# Monte Carlo dropout

本文介绍另一种很常见的神经网络不确定性量化方法，Monte Carlo Dropout，主要参考了以下资料：

- [Monte Carlo Dropout](https://towardsdatascience.com/monte-carlo-dropout-7fd52f8b6571)
- [Dropout as a Bayesian Approximation: Representing Model Uncertainty in Deep Learning](https://paperswithcode.com/paper/dropout-as-a-bayesian-approximation)
- [Monte Carlo dropout](https://docs.aws.amazon.com/prescriptive-guidance/latest/ml-quantifying-uncertainty/mc-dropout.html)
- [Tutorial: Dropout as Regularization and Bayesian Approximation](https://xuwd11.github.io/Dropout_Tutorial_in_PyTorch/)
- [Dropout as a Bayesian Approximation: Representing Model Uncertainty in Deep Learning](https://paperswithcode.com/paper/dropout-as-a-bayesian-approximation)

## Dropout

Monte Carlo dropout 首先需要神经网络具有dropout层，所以先简要介绍 Dropout。Dropout 会在每个训练步骤中简单地关闭一些神经元。每一步一组不同的神经元被关闭。从数学上讲，每个神经元都有被忽略的概率p，称为dropout rate。dropout rate通常设置在 0（无dropout）和 0.5（大约 50% 的所有神经元将被关闭）之间。确切值取决于网络类型、层大小以及网络过度拟合训练数据的程度。

![](img/1_YOOzhIJWi0IysZmprkrNMQ.png)

但是为什么要这样做呢？Dropout 是一种正则化技术，也就是说，它有助于防止过拟合。在数据很少和/或网络复杂的情况下，模型可能会记住训练数据，因此，在训练期间看到的数据上效果很好，但在新的、看不见的数据上却产生了糟糕的结果。这被称为过拟合，而 dropout 旨在缓解这种情况。

怎么做到的呢？有两种方法可以理解为什么关闭模型的某些部分可能是有益的。首先，信息在网络上的传播更加均匀。想想网络内部某处的单个神经元。还有一些其他神经元为其提供输入。使用 dropout，这些输入源中的每一个都可以在训练期间随时消失。因此，我们的神经元不能只依赖一两个输入，它必须分散其权重并关注所有输入。结果，它对输入变化变得不那么敏感，这导致模型更好地泛化。

从蒙特卡罗技巧的角度来看，对 dropout 有效性的另一种解释更为重要。由于在每次训练迭代中，都会随机采样每一层中要丢弃的神经元（根据该层的dropout rate），因此每次都会丢弃一组不同的神经元。因此，每次模型的架构都略有不同，所以可以将结果视为许多不同神经网络的平均集合，每个神经网络仅对一批数据进行训练。

最后一个细节：dropout 仅在训练期间使用。在推理时，也就是我们用我们的网络进行预测时，我们通常不应用任何 dropout——我们想要使用所有经过训练的神经元和连接。


## 蒙特卡洛

现在我们已经简单了解了dropout，那什么是蒙特卡洛？

在统计学中，蒙特卡罗是指一类依靠重复随机抽样来获得某个数值量分布的计算算法。

## Monte Carlo Dropout

由[Gal & Ghahramani (2016)](https://arxiv.org/abs/1506.02142)提出的 Monte Carlo Dropout是一聪明的认识，即使用常规 dropout 可以解释为众所周知的概率模型的贝叶斯近似：高斯过程。我们可以将许多不同的网络（丢弃了不同的神经元）视为来自所有可用模型空间的蒙特卡罗样本。这为推理模型的不确定性提供了数学依据，而且事实证明，这通常会提高其性能。

它是如何工作的？我们只是在测试时应用 dropout，仅此而已！然后，我们得到很多，而不是一个预测，每个模型一个。然后我们可以对它们进行平均或分析它们的分布。最好的部分是：它不需要对模型的架构进行任何更改。我们甚至可以在已经训练过的模型上使用这个技巧！

从理论上来说，首先我们想要一个预测的概率分布。

![](img/methods-2.png)

其中预测目标是y，输入是x，然后我们有N个训练样本 $\mathcal D = (x_i,y_i)_{i=1}^N$。当获取到预测的分布时，就能查看其变化和不确定性了。一种方法是需要学习一个函数分布或者参数的分布，即参数后验分布 $q(\Theta|\mathcal D)$

Monte Carlo (MC) dropout技术提供了一种可扩展的方式来学习预测的分布。MC dropout 的工作原理是随机关闭神经网络中的神经元，从而使网络正则化。每个 dropout 配置对应于来自近似参数后验分布$q(\Theta|\mathcal D)$的不同样本： 

$\Theta _t \sim q(\Theta|\mathcal D)$

其中 $\Theta _t$ 对应一个dropout配置，或者等价地说，从 估计的参数后验分布$q(\Theta|\mathcal D)$ （如下公式所示）采样的一个模拟。从近似后验$q(\Theta|\mathcal D)$可以实现模型似然的蒙特卡罗积分，从而揭示预测分布，如下所示：

![](img/methods-9.png)

为简单起见，可以假设似然是高斯分布的：

![](img/methods-10.png)

使用由均值和方差参数指定的高斯函数，这些参数是由 Monte Carlo dropout BNN 的模拟输出的：

![](img/methods-12.png)

下图说明了 MC dropout。每个 dropout 配置通过在每次前向传播中随机关闭（灰色圆圈）和打开（黑色圆圈）神经元来产生不同的输出。具有不同 dropout 配置的多次前向传递在均值 p(f(x, ø)) 上产生预测分布。

![](img/mc-dropout.png)

未完待续。。。 （TODO:补充code，参考[这里](https://github.com/xuwd11/Dropout_Tutorial_in_PyTorch)）